# Federated PyTorch TinyImageNet Tutorial

This notebook is an example of Transfer Learning 

Custom DataLoader is used with OpenFL Python API

In [1]:
#Install dependencies if not already installed
!pip install torch torchvision
%load_ext autoreload
%autoreload 2

You should consider upgrading via the '/home/itrushkin/.virtualenvs/openfl/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import glob
from torch.utils.data import Dataset, DataLoader
from PIL import Image

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import transforms as T

import openfl.native as fx
from openfl.federated import FederatedModel, FederatedDataSet

After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist')

Creating Workspace Directories
Creating Workspace Templates

New workspace directory structure:
workspace
├── tiny-imagenet-200.zip
├── kvasir.zip
├── plan
│   ├── defaults
│   │   ├── data_loader.yaml
│   │   ├── network.yaml
│   │   ├── tasks_keras.yaml
│   │   ├── collaborator.yaml
│   │   ├── assigner.yaml
│   │   ├── task_runner.yaml
│   │   ├── defaults
│   │   ├── tasks_tensorflow.yaml
│   │   ├── compression_pipeline.yaml
│   │   ├── aggregator.yaml
│   │   ├── tasks_torch.yaml
│   │   └── tasks_fast_estimator.yaml
│   ├── plan.yaml
│   ├── data.yaml
│   └── cols.yaml
├── logs
├── final_pytorch_model.h5
├── tensordb.csv
├── agg_to_col_two_signed_cert.zip
├── data
│   ├── segmented-images
│   │   ├── masks
│   │   ├── images
│   │   ├── license.txt
│   │   └── bounding-boxes.json
│   └── MNIST
│       ├── raw
│       └── processed
├── cert
│   ├── ca
│   │   ├── signing-ca.crt
│   │   ├── root-ca.crt
│   │   ├── signing-ca
│   │   ├── signing-ca.csr
│   │   └── root-ca
│   ├── s

The CSR Hash for file server/agg_nnlicv611.inn.intel.com.csr = 17772dccdb9657f78d9e1d192348c47b71282236762895577f4213c45494c078abb14b486be3c06de1facc9bff88ac7a
 Signing AGGREGATOR certificate
Creating COLLABORATOR certificate key pair with following settings: CN=one, SAN=DNS:one
  Moving COLLABORATOR certificate to: cert/col_one
The CSR Hash for file col_one.csr = 8ef6102125c312f3ca3c0f91321eab4919e8bfd9402e4ecd12f16b449e44f71bea1925d0fa7a4cda901be7006763950f
 Signing COLLABORATOR certificate

Registering one in plan/cols.yaml
Creating COLLABORATOR certificate key pair with following settings: CN=two, SAN=DNS:two
  Moving COLLABORATOR certificate to: cert/col_two
The CSR Hash for file col_two.csr = 0b2de658a76ec2021ce4e85a84d57ddb91ccc52f3380bb0241c1f5fd902a79cd0d9d934d01aa7ab0cf12eac34dd62fb1
 Signing COLLABORATOR certificate

Registering two in plan/cols.yaml


Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

#### Download the data

In [4]:
!wget --no-clobber http://cs231n.stanford.edu/tiny-imagenet-200.zip
!unzip -n tiny-imagenet-200.zip
TINY_IMAGENET_ROOT = './tiny-imagenet-200/'

File ‘tiny-imagenet-200.zip’ already there; not retrieving.

Archive:  tiny-imagenet-200.zip


#### Describe the dataset

In [5]:
class TinyImageNet(Dataset):
    """
    Contains 200 classes for training. Each class has 500 images. 
    Parameters
    ----------
    root: string
        Root directory including `train` and `val` subdirectories.
    split: string
        Indicating which split to return as a data set.
        Valid option: [`train`, `val`]
    transform: torchvision.transforms
        A (series) of valid transformation(s).
    """
    def __init__(self, root, split='train', transform=None, target_transform=None):
        NUM_IMAGES_PER_CLASS = 500
        self.root = os.path.expanduser(root)
        self.transform = transform
        self.target_transform = target_transform
        self.split_dir = os.path.join(self.root, split)
        self.image_paths = sorted(glob.iglob(os.path.join(self.split_dir, '**', '*.JPEG'), recursive=True))
        
        self.labels = {}  # fname - label number mapping

        # build class label - number mapping
        with open(os.path.join(self.root, 'wnids.txt'), 'r') as fp:
            self.label_texts = sorted([text.strip() for text in fp.readlines()])
        self.label_text_to_number = {text: i for i, text in enumerate(self.label_texts)}

        if split == 'train':
            for label_text, i in self.label_text_to_number.items():
                for cnt in range(NUM_IMAGES_PER_CLASS):
                    self.labels[f'{label_text}_{cnt}.JPEG'] = i
        elif split == 'val':
            with open(os.path.join(self.split_dir, 'val_annotations.txt'), 'r') as fp:
                for line in fp.readlines():
                    terms = line.split('\t')
                    file_name, label_text = terms[0], terms[1]
                    self.labels[file_name] = self.label_text_to_number[label_text]
                    
    
    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        file_path = self.image_paths[index]
        label = self.labels[os.path.basename(file_path)]
        label = self.target_transform(label) if self.target_transform else label
        return self.read_image(file_path), label

    def read_image(self, path):
        img = Image.open(path)
        return self.transform(img) if self.transform else img

def one_hot(labels, classes):
    return np.eye(classes)[labels]

In [6]:
normalize = T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

augmentation = T.RandomApply([
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.RandomResizedCrop(64)], p=.8)

training_transform = T.Compose([
    T.Lambda(lambda x: x.convert("RGB")),
    augmentation,
    T.ToTensor(),
    normalize])

valid_transform = T.Compose([
    T.Lambda(lambda x: x.convert("RGB")),
    T.ToTensor(),
    normalize])

#### Implement Federated dataset
We have to implement `split` method

In [7]:
from openfl.utilities.data import LogNormalPyTorchDatasetSplitter
from openfl.federated.data import PyTorchFederatedDataset

train_set = TinyImageNet(TINY_IMAGENET_ROOT, 'train', transform=training_transform)
valid_set = TinyImageNet(TINY_IMAGENET_ROOT, 'val', transform=valid_transform, \
                                      target_transform=lambda target: one_hot(target, 200))

fl_data = PyTorchFederatedDataset(train_set, valid_set, batch_size=32,
                                  data_splitter=LogNormalPyTorchDatasetSplitter(0, 2.0, 200))
num_classes = 200

#### Define model

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = torchvision.models.mobilenet_v2(pretrained=True)
        self.model.requires_grad_(False)
        self.model.classifier[1] = torch.nn.Linear(in_features=1280, \
                        out_features=num_classes, bias=True)

    def forward(self, x):
        x = self.model.forward(x)
        return x

    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.cross_entropy(input=output,target=target)

In [9]:
#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy, \
                        data_loader=fl_data)

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [10]:
collaborator_models = fl_model.setup(num_collaborators=10)
collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}

100%|██████████| 10/10 [00:00<00:00, 1006.24it/s]

Assigning 0:5 of 0 class to 0 col...
Assigning 5:10 of 1 class to 0 col...
Assigning 0:5 of 1 class to 1 col...
Assigning 5:10 of 2 class to 1 col...
Assigning 0:5 of 2 class to 2 col...
Assigning 5:10 of 3 class to 2 col...
Assigning 0:5 of 3 class to 3 col...
Assigning 5:10 of 4 class to 3 col...
Assigning 0:5 of 4 class to 4 col...
Assigning 5:10 of 5 class to 4 col...
Assigning 0:5 of 5 class to 5 col...
Assigning 5:10 of 6 class to 5 col...
Assigning 0:5 of 6 class to 6 col...
Assigning 5:10 of 7 class to 6 col...
Assigning 0:5 of 7 class to 7 col...
Assigning 5:10 of 8 class to 7 col...
Assigning 0:5 of 8 class to 8 col...
Assigning 5:10 of 9 class to 8 col...
Assigning 0:5 of 9 class to 9 col...
Assigning 5:10 of 10 class to 9 col...
Trying to append 336 of 0 class to 0 col...
Appending [  5   6   7   8   9  10  11  12  13  14  15  16  17  18  19  20  21  22
  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39  40
  41  42  43  44  45  46  47  48  49  50  51  52 


100%|██████████| 10/10 [00:00<00:00, 1614.62it/s]

Assigning 0:5 of 0 class to 0 col...
Assigning 5:10 of 1 class to 0 col...
Assigning 0:5 of 1 class to 1 col...
Assigning 5:10 of 2 class to 1 col...
Assigning 0:5 of 2 class to 2 col...
Assigning 5:10 of 3 class to 2 col...
Assigning 0:5 of 3 class to 3 col...
Assigning 5:10 of 4 class to 3 col...
Assigning 0:5 of 4 class to 4 col...
Assigning 5:10 of 5 class to 4 col...
Assigning 0:5 of 5 class to 5 col...
Assigning 5:10 of 6 class to 5 col...
Assigning 0:5 of 6 class to 6 col...
Assigning 5:10 of 7 class to 6 col...
Assigning 0:5 of 7 class to 7 col...
Assigning 5:10 of 8 class to 7 col...
Assigning 0:5 of 8 class to 8 col...
Assigning 5:10 of 9 class to 8 col...
Assigning 0:5 of 9 class to 9 col...
Assigning 5:10 of 10 class to 9 col...
Trying to append 38 of 0 class to 0 col...
Appending [1021 1165 1209 1395 1488 2164 2690 2828 2830 2901 3082 3506 3763 3955
 4061 4324 4541 5520 5630 5680 6124 6146 6445 6506 6528 6558 6766 6966
 6981 7702 7885 7998 8314 8420 8429 8734 8785 8788] of

In [11]:
#Original TinyImageNet dataset
print(f'Original training data size: {len(fl_data.train_set)}')
print(f'Original validation data size: {len(fl_data.valid_set)}\n')

#Collaborator one's data
for i, model in enumerate(collaborator_models):
    print(f'Collaborator {i}\'s training data size: {len(model.data_loader.train_set)}')
    print(f'Collaborator {i}\'s validation data size: {len(model.data_loader.valid_set)}\n')

#Collaborator three's data
#print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
#print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 100000
Original validation data size: 10000

Collaborator 0's training data size: 357
Collaborator 0's validation data size: 84

Collaborator 1's training data size: 495
Collaborator 1's validation data size: 46

Collaborator 2's training data size: 412
Collaborator 2's validation data size: 17

Collaborator 3's training data size: 74
Collaborator 3's validation data size: 10

Collaborator 4's training data size: 484
Collaborator 4's validation data size: 14

Collaborator 5's training data size: 10
Collaborator 5's validation data size: 10

Collaborator 6's training data size: 271
Collaborator 6's validation data size: 11

Collaborator 7's training data size: 11
Collaborator 7's validation data size: 31

Collaborator 8's training data size: 325
Collaborator 8's validation data size: 22

Collaborator 9's training data size: 349
Collaborator 9's validation data size: 42



In [12]:
#Run experiment, return trained FederatedModel
final_fl_model = fx.run_experiment(collaborators,{'aggregator.settings.rounds_to_train':10})

/home/itrushkin/.virtualenvs/openfl/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:281: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
/home/itrushkin/.virtualenvs/openfl/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:106: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


/home/itrushkin/.virtualenvs/openfl/lib/python3.8/site-packages/openfl/federated/task/runner_pt.py:457: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


In [13]:
#Save final model
final_fl_model.save_native('final_model.pth')